In [4]:
import basedosdados as bd

bd.list_datasets() # deve listar entre os datasets: 'br_anp_precos_combustiveis'

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=yV0TRmbJs2qxVprdMEfXjINh2xEa0t&prompt=consent&access_type=offline

dataset_id: 
	br_abrinq_oca 
-----------------------------------------------------------------------------------------------
dataset_id: 
	br_ana_atlas_esgotos 
-----------------------------------------------------------------------------------------------
dataset_id: 
	br_ana_reservatorios 
-----------------------------------------------------------------------------------------------
dataset_id: 
	br_anatel_banda_larga_fixa 
-----------------------------------------------------------------------------------------------
dataset_id: 
	br_anp_precos_combustiveis 
----------------------------------------------------

In [5]:
bd.list_dataset_tables('br_anp_precos_combustiveis') # deve listar 'microdados'


table_id: 
	microdados 
-----------------------------------------------------------------------------------------------



In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
BILLING_PROJECT_ID = os.getenv("BILLING_PROJECT_ID")

import basedosdados as bd

QUERY_PROJECT_ID = 'basedosdados'
DATASET_ID = 'br_anp_precos_combustiveis'
TABLE_ID = 'microdados'

def fetch_precos_combustiveis(ano):
    query = f"""
    SELECT *
    FROM `{QUERY_PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE ano = {ano}
    """
    return bd.read_sql(query, billing_project_id=BILLING_PROJECT_ID)

In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Crie uma sessão do Spark
spark = ( 
    SparkSession.builder.appName("Cassandra")
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/usr/local/sdkman/candidates/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a2ca0b3e-eb91-48ab-a46e-98c54a5a1083;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 i

In [12]:
from pyspark.sql.functions import when, lit, col

# Propriedades de conexão com o Cassandra
CASSANDRA_CONF = {
    "spark.cassandra.connection.host": "cassandra",
    "spark.cassandra.connection.port": 9042,
}

def ingestao_precos_combustiveis_ano(ano):
    print(':. Carregando o dataframe')
    df = fetch_precos_combustiveis(ano)

    print(':. Convertendo para o dataframe do Spark')
    df_spark = spark.createDataFrame(df)

    print(':. Tratando valores NaN')
    df_spark = (
        df_spark
        .withColumn('preco_compra', when(col('preco_compra') == 'NaN', lit(None)).otherwise(col('preco_compra')))
        .withColumn('preco_venda', when(col('preco_venda') == 'NaN', lit(None)).otherwise(col('preco_venda')))
    ) 

    print(':. Salvando no Cassandra')
    (
        df_spark.write.format("org.apache.spark.sql.cassandra")
        .options(**CASSANDRA_CONF)
        .options(keyspace="anp", table="precos_combustiveis_por_produto_e_uf")
        .mode("append")
        .save()
    )

In [13]:
for ano in range(2004, 2023):
    print(f'Ingestão de dados para o ano {ano}')
    ingestao_precos_combustiveis_ano(ano)


Ingestão de dados para o ano 2004
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


23/12/11 16:47:42 WARN TaskSetManager: Stage 0 contains a task of very large size (7893 KiB). The maximum recommended task size is 1000 KiB.


Ingestão de dados para o ano 2005
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


23/12/11 16:49:28 WARN TaskSetManager: Stage 1 contains a task of very large size (4300 KiB). The maximum recommended task size is 1000 KiB.


Ingestão de dados para o ano 2006
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


...

23/12/11 17:45:58 WARN TaskSetManager: Stage 16 contains a task of very large size (14642 KiB). The maximum recommended task size is 1000 KiB.


Ingestão de dados para o ano 2021
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


23/12/11 17:49:54 WARN TaskSetManager: Stage 17 contains a task of very large size (12635 KiB). The maximum recommended task size is 1000 KiB.


Ingestão de dados para o ano 2022
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


23/12/11 17:55:22 WARN TaskSetManager: Stage 18 contains a task of very large size (21561 KiB). The maximum recommended task size is 1000 KiB.
